In [6]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-07-01 11:52:40--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-01 11:52:40 (36.4 MB/s) - ‘minsearch.py.1’ saved [3832/3832]



In [7]:
import minsearch

In [8]:
import json 

In [9]:
with open('documents.json') as f_in:
    docs_raw = json.load(f_in)

In [10]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [13]:
documents[0]

In [14]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [16]:
SELECT * WHERE course = 'data-engineering-zoomcamp';

In [17]:
q = 'the course has already started, can I still join?'

In [ ]:
index.fit(documents)

In [ ]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5  
)

In [ ]:
results

In [ ]:
from openai import OpenAI

In [ ]:
client = OpenAI()

In [18]:
q 

'the course has already started, can I still join?'

In [1]:
response = client.chat.completions.create(
    model = 'gpt-4o',
    messages = [{"role": "user", "content": q}]
)

In [ ]:
response.choices[0].message.content

In [2]:
prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT. 
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE

    QUESTION: {question}

    CONTEXT: {context}
""".strip()

In [ ]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [ ]:
prompt = prompt_template.format(question=q, context=context).strip()

In [ ]:
response = client.chat.completions.create(
    model = 'gpt-4o',
    messages = [{"role": "user", "content": prompt}]
)

response.choices[0].message.content